Import kiểm tra data

In [ ]:
import pandas as pd

# Đọc dữ liệu
df = pd.read_csv("../data/Bengaluru_House_Data.csv")

# Hiển thị tất cả các cột
pd.set_option('display.max_columns', None)

# In ra các cột của dataset
print("Các cột của dataset:")
print(df.columns)

# In ra 5 dòng đầu của dữ liệu
print("\n5 dòng đầu của dữ liệu:")
print(df.head())

# Kiểm tra giá trị thiếu
print("\nGiá trị thiếu trong mỗi cột:")
print(df.isnull().sum())


ModuleNotFoundError: No module named 'pandas'

 Tiền xử lý dữ liệu

- Loại bỏ các dòng có giá trị thiếu trong cột `size` (vì đây là cột quan trọng để tách số phòng).
- Tách số phòng ngủ từ cột `size` (chuyển từ "2 BHK" thành số 2).
- Loại bỏ các dòng có giá trị không hợp lệ trong cột `total_sqft` (chỉ giữ lại các số hợp lệ).

In [ ]:
# Loại bỏ các dòng có giá trị NaN trong cột 'size'
df = df.dropna(subset=['size']).copy()

# Tách số phòng ngủ từ cột 'size' -> bhk
df['bhk'] = df['size'].str.extract(r'(\d+)').astype(int)

# Kiểm tra giá trị của cột 'total_sqft' và loại bỏ các dòng có giá trị không hợp lệ
sqft_str = df['total_sqft'].astype(str)
mask_numeric_sqft = sqft_str.str.match(r'^\d+(\.\d+)?$')
df_clean = df[mask_numeric_sqft].copy()
df_clean['total_sqft'] = df_clean['total_sqft'].astype(float)

# Chọn các biến đầu vào (features) và biến mục tiêu (target)
features = ['total_sqft', 'bhk', 'bath', 'balcony', 'location']
target = 'price'

# Tạo dataset mới với features + target, loại bỏ các dòng có giá trị thiếu
df_model = df_clean[features + [target]].dropna().copy()


print("5 dòng dữ liệu sau khi tiền xử lý:")
print(df_model.head())


5 dòng dữ liệu sau khi tiền xử lý:
   total_sqft  bhk  bath  balcony                  location   price
0      1056.0    2   2.0      1.0  Electronic City Phase II   39.07
1      2600.0    4   5.0      3.0          Chikka Tirupathi  120.00
2      1440.0    3   2.0      3.0               Uttarahalli   62.00
3      1521.0    3   3.0      1.0        Lingadheeranahalli   95.00
4      1200.0    2   2.0      1.0                  Kothanur   51.00


 Chia dữ liệu thành train/test (80/20)
- Sử dụng `train_test_split` để chia dữ liệu thành 80% cho huấn luyện (train) và 20% cho kiểm tra (test).

In [32]:
from sklearn.model_selection import train_test_split

X = df_model[features]
y = df_model[target]

# Chia dữ liệu thành train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# In kích thước của tập train và test
print("Kích thước tập train:", X_train.shape)
print("Kích thước tập test:", X_test.shape)


Kích thước tập train: (10019, 5)
Kích thước tập test: (2505, 5)


Xây dựng mô hình Linear Regression và Random Forest, sau đó so sánh RMSE
- Chúng ta sẽ xây dựng 2 mô hình: **Linear Regression** và **Random Forest Regressor**.
- Mã hóa dữ liệu phân loại `location` bằng **OneHotEncoder**.
- Sử dụng **RMSE** (Root Mean Squared Error) để đánh giá hiệu quả của mô hình

In [33]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Preprocessing: OneHotEncode cho 'location'
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['location']),
        ('num', 'passthrough', ['total_sqft', 'bhk', 'bath', 'balcony'])
    ]
)

# Linear Regression Model
linreg_model = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', LinearRegression())
])

# Random Forest Model
rf_model = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', RandomForestRegressor(n_estimators=200, random_state=42))
])

# Huấn luyện mô hình Linear Regression
linreg_model.fit(X_train, y_train)
y_pred_lin = linreg_model.predict(X_test)
rmse_lin = mean_squared_error(y_test, y_pred_lin) ** 0.5

# Huấn luyện mô hình Random Forest
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
rmse_rf = mean_squared_error(y_test, y_pred_rf) ** 0.5

# In kết quả
print("RMSE Linear Regression:", rmse_lin)
print("RMSE Random Forest   :", rmse_rf)

if rmse_rf < rmse_lin:
    print("\n➡ Random Forest dự đoán tốt hơn (RMSE thấp hơn).")
else:
    print("\n➡ Linear Regression dự đoán tốt hơn (RMSE thấp hơn).")


RMSE Linear Regression: 112.97331598007987
RMSE Random Forest   : 104.847705069759

➡ Random Forest dự đoán tốt hơn (RMSE thấp hơn).
